In [1]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
from datetime import datetime
from pprint import pprint
from bson import ObjectId

sys.path.append(str(Path("..").resolve()))
from src import *

# Create Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [2]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/5_optimized/omero_museum"

In [3]:
connector = MongoDBConnector("omero_museum", silent=True)
db = connector.db

### CR1: _Create a new comment to a specific artwork_

In [5]:
customer_id = db.visitors.find_one({"is_customer": True})["_id"]  # type: ignore
artwork_id = "Mona Lisa"
rating = 5
new_comment_id = ObjectId()

db.artworks.update_one(
    {"_id": artwork_id},
    {
        "$push": {
            f"comments_star_{rating}": {
                "_id": new_comment_id,
                "rating": rating,
                "date": datetime.now(),
                "message": "Capolavoro assoluto, emozionante da vedere dal vivo!",
            }
        }
    },
)
db.customers.update_one({"_id": customer_id}, {"$push": {"comments": new_comment_id}})

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

### CR2: _Create a new survey compilation_

In [ ]:
visitor_id = db.visitors.find_one({})["_id"]  # type: ignore

db.visitors.update_one(
    {"_id": visitor_id},
    {
        "$push": {
            "surveys": {
                "accompanying_persons_visit": "amici",
                "date_of_compilation": datetime(2025, 9, 5, 15, 30),
                "reason_for_visit": "Interesse per l'arte moderna",
                "number_of_visits": 2,
                "return": "sì",
                "type_of_visit": "visita guidata",
                "title_of_studies": "Diploma",
                "evaluation_of_experience": 9,
                "evaluation_of_facility": 8,
                "evaluation_of_visit": 10
            }
        }
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR3: _Create a new ticket for a visitor_

In [ ]:
visitor_id = db.visitors.find_one({})["_id"]  # type: ignore

db.visitors.update_one(
    {"_id": visitor_id},
    {
        "$push": {
            "tickets": {
                "date": datetime(2025, 9, 6, 11, 0),
                "price": 18.0,
                "check_in": False
            }
        }
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR4: _Create a new message_

In [ ]:
visitor_id = db.visitors.find_one({"chat": {"$exists": True}})["_id"]  # type: ignore

new_message = {
    "customerId": visitor_id,
    "sentDate": datetime(2025, 9, 5, 16, 0),
    "type": "text",
    "content": "Benvenuto al Museo Omero!",
    "deliveryStatus": "sent",
    "length": 24,
    "uri": None,
}

visitor = db.visitors.find_one({"_id": visitor_id})
chat = visitor["chat"]  # type: ignore
new_msgs = chat.get("new_messages", [])
old_msgs = chat.get("old_messages", [])

new_msgs.append(new_message)

# If there are more than 10 new messages, move the oldest to old messages
if len(new_msgs) > 10:
    oldest = new_msgs.pop(0)
    oldest_id = ObjectId()
    old_msgs.insert(0, oldest_id)
    db.messages.insert_one({"_id": oldest_id, **oldest})

db.visitors.update_one(
    {"_id": visitor_id},
    {"$set": {"chat.new_messages": new_msgs, "chat.old_messages": old_msgs}},
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR5: _Create a new trade_

In [6]:
artwork_id = "Mona Lisa"
customer_id = db.visitors.find_one({"is_customer": True})["_id"]  # type: ignore

db.artworks.update_one(
    {"_id": artwork_id},
    {
        "$set": {
            "trade": {
                "date_start": datetime(2025, 9, 10, 10, 0),
                "date_end": datetime(2025, 9, 15, 18, 0),
                "win_price": 12000,
                "winner_id": customer_id,
                "price_max": 15000,
                "price_min": 8000,
                "price_start": 5000,
                "min_increase": 200,
                "is_auction": True,
            }
        }
    },
)
db.visitors.update_one({"_id": customer_id}, {"$addToSet": {"tradeIds": "Mona Lisa"}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR6: _Create a new visitor_

In [7]:
db.visitors.insert_one({
    "isCustomer": False,
    "birthDate": datetime(1990, 5, 15, 2, 0),
    "gender": "male",
    "land": "Italy",
    "tickets": [
        {
            "date": datetime(2024, 7, 1, 10, 0),
            "price": 15.0,
            "check_in": True
        }
    ],
})


InsertOneResult(ObjectId('68bc891e9122b3cf6357d50c'), acknowledged=True)

### CR7: _Create a new activity_

In [ ]:
db.activities.insert_one({
    "room": "Rinascimentale",  
    "duration": 90,
    "enrolled": 5,
    "startDate": datetime(2025, 9, 4, 15, 0),
    "capacity": 30,
    "ticketIds": [34, 46, 9, 7, 12],
    "workshopTitle": "Pittura su tela"  
})


InsertOneResult(25, acknowledged=True)

### CR8: _Create a new artwork_

In [8]:
author_id = db.authors.find_one({})["_id"]  # type: ignore

db.artworks.insert_one(
    {
        "_id": "La Notte Stellata",
        "date": datetime(1889, 6, 1),
        "type": "painting",
        "description": "Opera celebre di Van Gogh",
        "isOriginal": True,
        "size": 73,
        "period": "Post-Impressionismo",
        "sellerId": 9,
        "donatorId": 13,
        "donationState": "refused",
        "locationName": "Parigi",
        "authorIds": [author_id],  # authors
        "tecniques": ["mosaic"],
        "materials": ["Glass", "Wood"],
        "comments_star_3": [
            {
                "rating": 3,
                "date": datetime(2025, 8, 12, 10, 30),
                "message": "Un buon lavoro, ma si può migliorare",
            }
        ],
    }
)

InsertOneResult('La Notte Stellata', acknowledged=True)

### CR9: _Create a new limited event_

In [16]:
author_id = db.authors.find_one({})["_id"]  # type: ignore
ticket_id = db.visitors.find_one({"tickets": {"$ne": None}})["tickets"][0]["_id"]  # type: ignore

db.limited_events.insert_one(
    {
        "_id": "The Future of Mosaic",
        "capacity": 40,
        "type": "exhibition",
        "startDate": datetime(2025, 10, 1, 10, 0),
        "endDate": datetime(2025, 10, 1, 18, 0),
        "theme": "Innovation",
        "description": "Mostra temporanea di opere moderne",
        "roomName": "Contemporaneo",  # room id
        "authorIds": [author_id],
        "artworkTitles": ["The Scream", "Guernica"],
        "ticketIds": [42, 51, 8, 6, 7],
    }
)

InsertOneResult('The Future of Mosaic', acknowledged=True)

### CR10: _Create a new employee_

In [19]:
out=db.roles.insert_one(
    {
        "phone_number": "+39 0922684697",
        "surname": "Gozzano",
        "curriculum": "http://monteverdi.com/",
        "birth_date": datetime(2025, 8, 23, 7, 6, 2),
        "date_start": datetime(2025, 9, 2, 18, 14),
        "email": "carmelo87@example.com",
        "hometown": "Trevenzuolo",
        "name": "Amedeo",
        "gender": "F",
        "username": "ludovico03",
        "password": "67d71a0a46db3ea107aa55cab0f8de7d",
        "salary": 40000,
        "desk": 2,
        "department": "Amministrazione",
        "type": "teaching staff",
        "english_cert": "A1",
    }
)

### CR11: _Create a new shift_ (correction json first)

In [22]:
db.roles.update_one(
    {"_id": out.inserted_id},
    {
        "$push": {
            "shifts": {
                "time_start": "08:50:23",
                "time_end": "16:58:45",
                "day_of_week": "martedì",
            }
        }
    },
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR12: _Create a new author_

In [ ]:
db.authors.insert_one({
    "surname": "Rossi",
    "birth_date": datetime(1975, 4, 12),
    "home_town": "Milano",
    "name": "Giovanni",
    "gender": "M"
})

InsertOneResult(ObjectId('68bab0df278cb5df47bf676b'), acknowledged=True)

### CR13: _Create a new workshop_

In [ ]:
db.workshops.insert_one({
    "_id": "Pittura su tela",
    "duration": datetime(1970, 1, 1, 2, 0),         
    "price_class": 80.00,        
    "price_person": 25.00,      
    "type": "Design"  
})

InsertOneResult('Pittura su tela', acknowledged=True)

### CR14: _Create a new supplier_

In [ ]:
db.suppliers.insert_one(
    {
        "name": "Maurizio Conte",
        "iban": "IT60X0542811101000000123456",
        "isState": True,
        "isMuseum": False,
        "email": "conte_mauto@outlook.it",
        "phoneNum": "+390276543210",
    }
)

InsertOneResult(25, acknowledged=True)